In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from tssb.utils import load_time_series_segmentation_datasets
from tssb.evaluation import covering

In [21]:
# loading datasets
tssb = load_time_series_segmentation_datasets(names=['UWaveGestureLibraryY']) 
# names=["ArrowHead", "InsectWingbeatSound", "TwoLeadECG"]
tssb.shape

(1, 4)

In [22]:
tssb.head(10)

,dataset,window_size,change_points,time_series
0,UWaveGestureLibraryY,10,"[600, 1131, 1652, 2193]","[-1.523467, -0.106794, 0.190361, 0.979247, 0.3..."


In [23]:
from CLaSP import CLaSP
from copy import deepcopy

In [6]:
# for i in range(75):
#     name = tssb.iloc[i,::]['dataset']
#     wind_size = tssb.iloc[i,::]['window_size']
#     true_chp = tssb.iloc[i,::]['change_points']      
#     time_series = tssb.iloc[i,::]['time_series'] 
#     if time_series.shape[0] > 4500:
#         continue
#     print(name, ' ', len(time_series), ' ', len(true_chp))

In [24]:
res_knn = {}
for i in range(tssb.shape[0]):
    name = tssb.iloc[i,::]['dataset']
    wind_size = tssb.iloc[i,::]['window_size']
    true_chp = tssb.iloc[i,::]['change_points']      
    time_series = tssb.iloc[i,::]['time_series']  
    if time_series.shape[0] > 4500:
        continue
    
    cls = CLaSP(window=wind_size, solver='knn')
    cls.fit(time_series[::2], n_segments=len(true_chp)+1)
    
    numbers = set()
    for elem in cls.stack:
        numbers.add(elem['start'])
        numbers.add(elem['stop'])    
    res_knn[name] = np.array(sorted(list(numbers))[1:-1])*2
    print(name)
    print('true: ',true_chp)
    print('pred: ',np.array(sorted(list(numbers))[1:-1])*2)
    print('='*100)

100%|████████████████████████████████████████████████████████████████████████████████| 245/245 [00:02<00:00, 93.53it/s]

UWaveGestureLibraryY
true:  [ 600 1131 1652 2193]
pred:  [ 476 1116 1840 2316]


In [7]:
res_svd = {}
for i in range(tssb.shape[0]):
    name = tssb.iloc[i,::]['dataset']
    wind_size = tssb.iloc[i,::]['window_size']
    true_chp = tssb.iloc[i,::]['change_points']      
    time_series = tssb.iloc[i,::]['time_series']  
    if time_series.shape[0] > 4500:
        continue
    
    cls = CLaSP(window=wind_size, solver='svd')
    cls.fit(time_series[::2], n_segments=len(true_chp)+1)
    
    numbers = set()
    for elem in cls.stack:
        numbers.add(elem['start'])
        numbers.add(elem['stop'])    
    res_svd[name] = np.array(sorted(list(numbers))[1:-1])*2
    print(name)
    print(true_chp)
    print(np.array(sorted(list(numbers))[1:-1])*2)
    print('='*100)

100%|████████████████████████████████████████████████████████████████████████████████| 889/889 [01:04<00:00, 13.74it/s]


CricketZ
[ 600 1106 1574 2174]
[ 396  604  628 2394]


In [ ]:
# import pickle
# with open('svd_scores.pickle', 'wb') as handle:
#     pickle.dump(res_svd, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
import pickle
with open('knn_scores.pickle', 'rb') as handle:
    res_knn = pickle.load(handle)

In [20]:
for key in res_knn.keys():
    true_chp = tssb.loc[tssb['dataset'] == key]['change_points'].values[0]
    pred_chp = res_knn[key]
    for i in range(len(pred_chp)):
        diff = np.abs(pred_chp[i] - true_chp[i])
        if abs(diff) > 500:
            print(key)
            print('True:',true_chp)
            print('Pred:',pred_chp)  
            print('='*20)

DistalPhalanxTW
True: [ 580 2220]
Pred: [262 584]
SonyAIBORobotSurface1
True: [420]
Pred: [1046]
UWaveGestureLibraryX
True: [ 600 1131 1652 2193]
Pred: [ 634 1126 2188 2510]
UWaveGestureLibraryY
True: [ 600 1131 1652 2193]
Pred: [ 104  580 1150 2242]
UWaveGestureLibraryY
True: [ 600 1131 1652 2193]
Pred: [ 104  580 1150 2242]
UWaveGestureLibraryZ
True: [ 600 1131 1652 2193]
Pred: [ 626 1080 1122 2198]
WordSynonyms
True: [ 202 2227]
Pred: [ 976 2188]


In [15]:
res_knn['SonyAIBORobotSurface1']

array([262, 584], dtype=int64)

In [19]:
tssb.loc[tssb['dataset'] == 'SonyAIBORobotSurface1']['change_points'].values[0]

array([420])

In [ ]:
diff_list_svd = []
for key in res_svd.keys():
    true_chp = tssb.loc[tssb['dataset'] == key]['change_points'].values[0]
    pred_chp = res_svd[key]
    for i in range(len(pred_chp)):
        diff = np.abs(pred_chp[i] - true_chp[i])
        if abs(diff) < 500:
            diff_list_svd.append(diff)

In [ ]:
diff_list_knn = []
for key in res_knn.keys():
    true_chp = tssb.loc[tssb['dataset'] == key]['change_points'].values[0]
    pred_chp = res_knn[key]
    for i in range(len(pred_chp)):
        diff = np.abs(pred_chp[i] - true_chp[i])
        if abs(diff) < 500:
            diff_list_knn.append(diff)

In [ ]:
plt.hist(diff_list_svd, bins=20, alpha=0.5)
plt.hist(diff_list_knn, bins=20, alpha=0.5)
plt.show()

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(6,4), sharey=True)
axs[0].boxplot(diff_list_svd)
axs[1].boxplot(diff_list_knn)

axs[0].set_title('SVD')
axs[1].set_title('KNN')
plt.show()

In [ ]:
line_ds = tssb.loc[tssb['dataset'] == 'InsectWingbeatSound']
line_ts = line_ds['time_series'].values[0]
line_chp = line_ds['change_points'].values[0]
line_chp

In [ ]:
plt.figure(figsize=(8,3))
cur_idx = 0
for i in range(len(line_chp)):
    plt.plot(np.arange(cur_idx,line_chp[i]), line_ts[cur_idx:line_chp[i]])
    cur_idx = line_chp[i]
plt.plot(np.arange(cur_idx,len(line_ts)-1), line_ts[cur_idx:len(line_ts)-1])
plt.show()

In [ ]:
from CLaSP import CLaSP

In [ ]:
cls = CLaSP(window=10, solver='knn')

In [ ]:
cls.fit(line_ts, n_segments=4)

In [ ]:
cls.stack

In [ ]:
numbers = set()
for elem in cls.stack:
    numbers.add(elem['start'])
    numbers.add(elem['stop'])    

In [ ]:
numbers

In [ ]:
plt.imshow(cls.X_dist, cmap='seismic')
plt.show()